In [2]:
import os
import json
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

from typing import *
from matplotlib import pyplot as plt
import plotly.express as px

if os.getcwd().endswith('notebooks'):
    os.chdir('..')

In [3]:
raw_data_filename = 'DataSet_EKB_200000.xlsx'

df = pd.read_excel(
    raw_data_filename,
    engine='openpyxl',
)

# df = df.head(10000)

df['Другая продукция в контрактах'] = (
    df['Другая продукция в контрактах']
    .progress_apply(
        lambda x: (
            x.rsplit('{', maxsplit=1)[0][:-1] + ']'
            if not x.endswith(']') else x
        )
        if not pd.isnull(x) else None
    )
)



100%|██████████| 200000/200000 [00:01<00:00, 193371.98it/s]


In [4]:
def parse_json(s: str) -> list:
    if not pd.notnull(s):
        result = None
    else:
        try:
            result = json.loads(s)
        except json.JSONDecodeError:
            return None

    return result


def prepare_data(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    df['Другая продукция в контрактах'] = df['Другая продукция в контрактах'].progress_apply(parse_json)
    df['cat'] = df['Код КПГЗ'].apply(lambda x: '.'.join(str(x).split('.')[:4]))

    df_train = df.tail(180000)
    df_test = df.head(20000)

    df_test['Другая продукция в контрактах'] = (
        df_test['Другая продукция в контрактах']
        .apply(lambda l: [str(x['OtherSkuId']) for x in l] if l is not None else [])
    )

    return df_train, df_test

In [5]:
def calc_recall(
    y_true: List[List[str]],
    y_pred: List[List[str]],
) -> float:
    n_relevant = 0
    n_total = 0
    for true, pred in zip(y_true, y_pred):
        n_relevant += len([item for item in true if item in pred])
        n_total += len(true)

    recall = n_relevant / n_total

    return recall


def calc_precision(
    y_true: List[List[str]],
    y_pred: List[List[str]],
) -> float:
    n_found = 0
    n_total = len(y_true)
    for true, pred in zip(y_true, y_pred):
        n_found += any([item for item in pred if item in true])

    precision = n_found / n_total

    return precision


def metrics(
    y_true: List[List[str]],
    y_pred: List[List[str]],
) -> Tuple[float, float]:
    recall = calc_recall(y_true, y_pred)
    precision = calc_precision(y_true, y_pred)
    return recall, precision


In [6]:
def get_pred_from_baseline(df_train: pd.DataFrame, df_test: pd.DataFrame) -> List[List[str]]:
    df_train['Кол-во заключенных контрактов'] = df_train['Кол-во заключенных контрактов'].fillna(0)
    df_train['tuple'] = df_train.apply(lambda row: (row['Идентификатор СТЕ'], row['Кол-во заключенных контрактов']), axis=1)
    df_cat = (
        df_train
        .groupby('cat')['tuple']
        .apply(lambda group: [x[0] for x in sorted(list(group), key=lambda x: -x[1])][:20])
    )

    y_pred_baseline = (
        df_test
        .join(
            df_cat,
            how='left',
            on='cat'
        )
        ['tuple']
        .apply(lambda d: d if isinstance(d, list) else [])
        .apply(lambda l: [str(x) for x in l][:20])
        .values
    )

    return y_pred_baseline


def get_pred_from_model(df_test: pd.DataFrame, df_pred: pd.DataFrame) -> List[List[str]]:
    df_pred['Сопутствующие товары'] = df_pred['Сопутствующие товары'].apply(lambda x: x[1:-1].split())

    df_pred_ = (
        df_test
        .set_index('Идентификатор СТЕ')
        .join(
            df_pred
            .set_index('Идентификатор СТЕ'),
            how='left'
        )
    )
    df_pred_['Сопутствующие товары'] = df_pred_['Сопутствующие товары'].apply(lambda d: d if isinstance(d, list) else [])

    y_pred_from_model = df_pred_['Сопутствующие товары'].values

    return y_pred_from_model

In [7]:
df_train, df_test = prepare_data(df)

100%|██████████| 200000/200000 [00:25<00:00, 7896.92it/s] 
<ipython-input-4-d381978901cd>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Другая продукция в контрактах'] = (


In [8]:
df_pred = pd.read_csv('result_similars.csv', index_col=0)
y_true = df_test['Другая продукция в контрактах'].values
#y_pred_from_baseline = get_pred_from_baseline(df_train, df_test)
y_pred_from_model = get_pred_from_model(df_test, df_pred)
metrics(y_true, y_pred_from_model)

(0.005926794653465269, 0.27905)

/var/folders/_5/bdhj5kd96qb1ll895tjts0sm0000gq/T/ipykernel_72813/435196024.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Кол-во заключенных контрактов'] = df_train['Кол-во заключенных контрактов'].fillna(0)
/var/folders/_5/bdhj5kd96qb1ll895tjts0sm0000gq/T/ipykernel_72813/435196024.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['tuple'] = df_train.apply(lambda row: (row['Идентификатор СТЕ'], row['Кол-во заключенных контрактов']), axis=1)


In [12]:
metrics(y_true, y_pred_from_baseline)

(0.006726597512868572, 0.34175)

(0.0050806150189221175, 0.29005)

-------

In [162]:
df_test.index[5]

1228720

In [175]:
df['Идентификатор СТЕ'] = df['Идентификатор СТЕ'].astype(str)

In [178]:
df.set_index('Идентификатор СТЕ').loc[str(df_test.index[5])][['Наименование СТЕ', 'Категория']].values

array(['Блокнот на спирали А5 60 листов 40 штук в упаковке', 'Блокнот'],
      dtype=object)

In [181]:
[x['OtherSkuName'] for x in df.set_index('Идентификатор СТЕ').loc[str(df_test.index[5])]['Другая продукция в контрактах']]

['ArtSpace Альбом для акварели 30 листов вид 1',
 'Avery Zweckform Этикетки для кухни Living 47,5 х 73 мм',
 'Berlingo Корректирующая жидкость 20 мл',
 'Berlingo Папка-скоросшиватель цвет фиолетовый 20 шт',
 'Berlingo Ручка шариковая Mega Soft синяя',
 'Berlingo Текстовыделитель цвет розовый',
 'BIC Увлажнитель для пальцев гелевый 1314670',
 'Brauberg Нож канцелярский 18 мм 235402',
 'Centropen Набор перманентных маркеров 4 цвета',
 'Colop Сменная штемпельная подушка E/R45 цвет синий',
 'Crown Ручка гелевая HJR-500R зеленая',
 'Faber-Castell Текстовыделитель цвет зеленый',
 'Hatber Тетрадь Mono Colour 96 листов в клетку цвет синий',
 'Index Ежедневник Altero недатированный 168 листов',
 'Kroyter Альбом для рисования акварелью 40 листов',
 "Maped Карандаш чернографитный Black Pep's",
 'Maped Нож канцелярский цвет голубой 9 мм',
 'Milan Ластик 1012',
 'Milan Ластик 118 цвет белый',
 'Milan Ластик Extra Soft 5020 прямоугольный',
 'Milan Ластик Nata 648 прямоугольный',
 'MunHwa Ручка шарик

In [179]:
df.set_index('Идентификатор СТЕ').loc[y_pred_baseline[5]]['Наименование СТЕ']

Идентификатор СТЕ
1252158     Бумага для флипчартов Блок бумаги для флипчарт...
1209261     Блокнот для флипчарта BRAUBERG, 50 листов, чис...
1223616     Блокнот А5, 80 л., гребень, мелованный картон ...
1226194     Бумага для флипчартов Блок бумаги для флипчарт...
23454650    Тетрадь 96л., А4, клетка OfficeSpace, бумвинил...
22264995    Бизнес-тетрадь Attache Selection Fluid A4 96 л...
1209258     Блокнот для флипчарта BRAUBERG, 20 листов, кле...
16892261    Блокнот Полином Природа А5 80 листов разноцвет...
22993304    Блокнот А5 (146?205 мм), 80 л., гребень, карто...
28533123    Бумага для флипчартов 67.5х98 см белая 20 лист...
1209188     Блокноты для флипчарта ОФИСМАГ, комплект 5 шт....
1223642     Блокнот А5, 80 л., гребень, жесткая подложка, ...
1208848     Блокноты для флипчарта BRAUBERG, комплект 5 шт...
1223615     Блокнот А4, 80 л., спираль, жесткая подложка, ...
1208845     Блокнот для флипчарта "2х3" ("Дважды три", Пол...
1230383     Бумага для флипчартов Блок бумаги для фл

In [174]:
df.set_index('Идентификатор СТЕ').loc[y_pred_good[5]]['Наименование СТЕ']

Идентификатор СТЕ
28533123    Бумага для флипчартов 67.5х98 см белая 20 лист...
1236648     Блокнот на спирали А5 60л. синий картон д/лог....
1228731     Блокнот на спирали А4 80л. ATTACHE клетка, одн...
1228768          Блокнот А5 60л. ULTIMATE BASICS, ACTIVE BOOK
1252718       Блокнот А6,пласт обл,80л, ATTACHE FANTASY,салат
20807500    Булавки Attache Economy металлические 24 мм (5...
1232159     Булавки металлич. 30 мм ATTACHE Б35, 500шт/уп....
1248232     Булавки металлич. 30 мм ATTACHE Б35, 500шт/уп....
1232348                                Бокс для бумаги 9х9х9 
1232347                                Бокс для бумаги 9х9х5 
20290603    Булавки для пробковых досок Globus (30 мм, 150...
23368841    Булавки для пробковых досок Globus (30 мм, 100...
Name: Наименование СТЕ, dtype: object